In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random


na = ['?']
tmp = ["symboling","normalized-losses","make","fuel-type", "aspiration", "num-of-doors", "body-style", \
       "drive-wheels","engine-location", "wheel-base", "length", "width", "height", "curb-weight", "engine-type", "num-of-cylinders", \
           "engine-size", "fuel-system", "bore", "stroke", "compression-ratio", "horsepower",  "peak-rpm", "city-mpg", "highway-mpg", \
               "price"]



df = pd.read_csv(r"/home/joker/Downloads/Video/SEM1-Downloads/CS771A/assignments/ass1_graded/imports-85.data", header = None, na_values = na,  names= tmp)
#df.columns = tmp
df = df.replace('?', np.nan)

df.fillna(df.mean(), inplace=True)

mode = str(df['num-of-doors'].mode())
df['num-of-doors'].fillna("four", inplace=True)

df.isna().sum()


symboling            0
normalized-losses    0
make                 0
fuel-type            0
aspiration           0
num-of-doors         0
body-style           0
drive-wheels         0
engine-location      0
wheel-base           0
length               0
width                0
height               0
curb-weight          0
engine-type          0
num-of-cylinders     0
engine-size          0
fuel-system          0
bore                 0
stroke               0
compression-ratio    0
horsepower           0
peak-rpm             0
city-mpg             0
highway-mpg          0
price                0
dtype: int64

In [2]:

a = list(df.columns.values)

dummy_cols = [col for col in df.columns if (df.dtypes[col] != np.float64 and df.dtypes[col] != np.int64)]
df = pd.get_dummies(df, columns = dummy_cols)


for d in dummy_cols:
    a.remove(d)

a.remove("price")
    
for col in a:
    maxx = df[col].max()
    minn = df[col].min()
    
    
    df[col] = df[col].apply(lambda x: (x-minn)/(maxx-minn))
    
training_data_u = df.sample(frac=0.8, random_state=25)
testing_data_u = df.drop(training_data_u.index)

validation_data_u = training_data_u.sample(frac=0.25, random_state=25)
training_data_u = training_data_u.drop(validation_data_u.index)

In [3]:
training_data_u

,symboling,normalized-losses,wheel-base,length,width,height,curb-weight,engine-size,bore,stroke,...,num-of-cylinders_twelve,num-of-cylinders_two,fuel-system_1bbl,fuel-system_2bbl,fuel-system_4bbl,fuel-system_idi,fuel-system_mfi,fuel-system_mpfi,fuel-system_spdi,fuel-system_spfi
26,0.6,0.434555,0.206997,0.241791,0.291667,0.233333,0.194337,0.109434,0.307143,0.552381,...,0,0,0,1,0,0,0,0,0,0
122,0.6,0.465969,0.206997,0.391045,0.291667,0.250000,0.272692,0.139623,0.307143,0.552381,...,0,0,0,1,0,0,0,0,0,0
190,1.0,1.000000,0.230321,0.367164,0.308333,0.300000,0.284329,0.181132,0.464286,0.633333,...,0,0,0,0,0,0,0,1,0,0
43,0.4,0.298429,0.224490,0.441791,0.125000,0.475000,0.329325,0.188679,0.550000,0.552381,...,0,0,0,1,0,0,0,0,0,0
69,0.4,0.146597,0.586006,0.692537,0.833333,0.591667,0.778510,0.460377,0.742857,0.747619,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,0.8,0.298429,0.384840,0.540299,0.500000,0.441667,0.395268,0.283019,0.464286,0.633333,...,0,0,0,0,0,0,0,1,0,0
130,0.4,0.298429,0.276968,0.602985,0.516667,0.616667,0.423196,0.267925,0.657143,0.871429,...,0,0,0,0,0,0,0,1,0,0
116,0.4,0.502618,0.620991,0.680597,0.675000,0.741667,0.684251,0.343396,0.828571,0.690476,...,0,0,0,0,0,1,0,0,0,0
158,0.4,0.136126,0.265306,0.376119,0.341667,0.433333,0.305275,0.184906,0.521429,0.609524,...,0,0,0,0,0,1,0,0,0,0


In [6]:
tmp = df["price"]
df.drop("price", axis=1, inplace=True)
df['price'] = tmp

column_names = []
for i in df.columns.values:
    column_names.append(i)

In [7]:
def cross_validate():
# 10 fold cross validation since we are using 10 samples for each k
    final = []
    for k in range(1,15):
        
        rmse = 0
        for r in random.sample(range(20,40), 10):
    
            training_data = df.sample(frac=0.8, random_state=r)
            testing_data = df.drop(training_data.index)
            
            validation_data = training_data.sample(frac=0.25, random_state=r)
            training_data = training_data.drop(validation_data.index)
            
            
            def getPrice(x,ind,k):
                
                out = []
                for i in range(len(training_data)):
                    d = {}
                    d['price'] = training_data.iloc[i, 75]
                    d['distance'] = np.linalg.norm(np.array(x.iloc[ind, 0:75])-np.array(training_data.iloc[i,0:75]), ord=2)
                    
                    
                    out.append(d)
                
                out = sorted(out, key=lambda x:x['distance'])[:k]   
                
                mean = sum(d['price'] for d in out)/k
                return mean
            
            
            def knn(testing, k):
               mse_sum = 0
               for i in range(len(testing)):
                   #print(np.abs(getPrice(testing, i, k) - testing.iloc[i, 15]))
                   mse_sum +=np.abs(getPrice(testing, i, k) - testing.iloc[i, 75])**2
               return np.sqrt(mse_sum/len(testing))               
    
            '''
            
            print(k, knn(testing_data, k))
            rmse["rmse"] = knn(testing_data, k)
            rmse["k"] = k
            final.append(rmse)
            '''
            rmse = rmse + knn(validation_data, k)
            
                
        #
        
        #rmse_df.plot(y='rmse', x='k')
        rmse1 = {}
        rmse_avg = rmse/10
        print(k, rmse_avg)
        rmse1["rmse_avg"] = rmse_avg
        rmse1["k"] = k
        final.append(rmse1)
    
    
    
    rmse_df = pd.DataFrame(final)     
    rmse_df.dtypes
    
    rmse_df.plot(y = "rmse_avg", x = "k")
    
    

In [ ]:
cross_validate()

1 4054.224894977028
2 3720.0023269695857
3 3325.096756249739
4 3979.9281038018285
5 4255.167702242385
6 4508.214450616137


In [ ]:
def sub_lists (l):
    lists = [()]
    for i in range(len(l) + 1):
        for j in range(i):
            lists.append(set(l[j: i]))
    return lists
    
column_idx = [i for i in range(0,75)]
#column_names.remove('price')
perms = sub_lists(column_idx)[1:]

for p in perms:
    p.add(75)

In [ ]:
def getPrice1(x,ind,k, cols):
    #print("gp"*10)  
    
    
    #print( cols)
    out = []
    for i in range(len(training_data)):
        d = {}
        d['price'] = training_data.iloc[i, len(cols)-1]
        #print(d['price'], cols)
        d['distance'] = np.linalg.norm(np.array(x.iloc[ind, len(cols)-1])-np.array(training_data_u.iloc[i, cols]), ord=2)
        
        
        out.append(d)
    
    out = sorted(out, key=lambda x:x['distance'])[:k]   
    
    mean = sum(d['price'] for d in out)/k
    return mean


def knn1(testing, k, cols):
   mse_sum = 0
   #print(cols)
   for i in range(len(testing)):
       #print(np.abs(getPrice(testing, i, k) - testing.iloc[i, 15]))
       mse_sum +=np.abs(getPrice1(testing, i, k, cols) - testing.iloc[i, len(cols)-1])**2
     
   #print("@"*100)     
   return np.sqrt(mse_sum/len(testing))   

k = 3 #learnt from CV


In [ ]:
final_cols = []


for i in range(len(perms)):
    #print(perms[i])
    df1 = df.iloc[:,list(perms[i])]
    #print(df1.head())
    #print(df1.head())
    '''
    b = len(df1.columns)-1
    print(b)
    tmp = df1.iloc[:, b]
    df1.drop(df1.columns[[b]], axis=1, inplace=True)
    df1.iloc[:, b] = tmp
    
    break 
    '''   
    training_data = df1.sample(frac=0.8, random_state=25)
    testing_data = df1.drop(training_data.index)
    
    validation_data = training_data.sample(frac=0.25, random_state=25)
    training_data = training_data.drop(validation_data.index)
    
    #print(training_data.head())
    #print(testing_data.head())
    #print(validation_data.head())

    
    #print("-"*100)
    alpha = {}
    alpha['columns'] = perms[i]
    alpha['rmse'] = knn1(validation_data, k, list(perms[i]))
    final_cols.append(alpha)
    print(i+1)
    
    


In [ ]:
l0_df = pd.DataFrame(final_cols)
